Predict label for test dataset


In [ ]:

!pip3 install scispacy
!pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import spacy
import scispacy
import en_core_sci_lg
from scispacy.umls_linking import UmlsEntityLinker
import re
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


!pip install transformers
import torch

import configparser,os,time,random,pickle
import numpy as np

import torch.nn.functional as F

#learning_rate=0.00001
#max_number_of_epochs=4
#gradient_clipping=1
#patience=3
batch_size=16
device=torch.device('cuda')

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have", "da": "the", "w/o": "without","w/": "with"}

def camel_case_split(sent):
  list_str = []
  for word in sent.split(' '):
    if word and word[0]=='#':
      regex = r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))'
      if re.search(regex, word):
        camel_case = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', word)
        for camel_case_element in camel_case:
          list_str.append(camel_case_element)
    else:
        list_str.append(word)
  sent = ' '.join(list_str)
  return sent  

def textprocess(data_path):
  textprocess_list = []
  with open(data_path,'rt',encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
      items = line.strip().split('\t')
      items[2] = re.sub('http://\S+|https://\S+', '', items[2])
      items[2] = re.sub(' rt[ ]@[\S]+','',items[2]) #remove Retweet @[Name]
      items[2] = re.sub('@[\S]+','',items[2]) # Remove @[Name]
      items[2] = re.sub('_[\S]?','',items[2]) #remove underscore
      items[2] = re.sub('&amp;?','and',items[2]) #replace '&amp' by 'and'
      items[2] = re.sub('&lt;','<',items[2]) #replace '&lt;' by '<'
      items[2] = re.sub('&gt;','>',items[2]) #replace '&gt;' by '>'
      items[2] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in items[2]] #Remove non-ascii words or characters
      items[2] = ''.join([str(elem) for elem in items[2]])
      items[2]= re.sub('([.,!?()])', r' \1 ',items[2]) #add a space between a word and a punctuation.
      items[2] = re.sub('\s{2,}', ' ', items[2]) #add a space between a word and a punctuation.
      items[2] = camel_case_split(items[2])
      items[2] = re.sub('#','',items[2]) #remove '#' tagging
      items[2] = items[2].lower() #lowercase
      items[2] = items[2].strip() #remove whitespaces at the begining and the end of tweet
      items[2] = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in items[2].split(' ')])
      textprocess_list.append([items[0],items[2]])
  return textprocess_list

def mednorm(df):
  nlp = en_core_sci_lg.load()
  linker = UmlsEntityLinker(threshold=0.7)
  nlp.add_pipe(linker)
  mednorm_list = []
  for row in df.itertuples(index=False):
    doc = nlp(row[1])  
    has_umls = False
    for entity in doc.ents:
      if len(entity._.umls_ents) != 0:
        has_umls = True
        break
    if has_umls: 
      mednorm_list.append([row[0],row[1]])
  return mednorm_list

def preprocess(df):
  df.encoded_tokens = [tokenizer.encode_plus(text,add_special_tokens=True)['input_ids'] for text in df['tweet']] #encoded tokens for each tweet
  df.attention_mask = [tokenizer.encode_plus(text,add_special_tokens=True)['attention_mask'] for text in df['tweet']]
  encoded_tokens = list(df.encoded_tokens)
  attention_mask = list(df.attention_mask)
  return encoded_tokens,attention_mask

# Convert indices to Torch tensor and dump into cuda
def feed_generator(encoded_tokens,attention_mask):

    batch_size = 16
    batch_seq = [x for x in range(int(len(encoded_tokens)/batch_size))]


    shuffled_encoded_tokens,shuffled_attention_mask = encoded_tokens,attention_mask

    res = len(encoded_tokens)%batch_size
    if res != 0:
        batch_seq = [x for x in range(int(len(encoded_tokens)/batch_size)+1)]
    shuffled_encoded_tokens = shuffled_encoded_tokens+shuffled_encoded_tokens[:res]
    shuffled_attention_mask = shuffled_attention_mask+shuffled_attention_mask[:res]

    for batch in batch_seq:
        maxlen_sent = max([len(i) for i in shuffled_encoded_tokens[batch*batch_size:(batch+1)*batch_size]])
        token_tensor = torch.tensor([tokens+[0]*(maxlen_sent-len(tokens)) for tokens in shuffled_encoded_tokens[batch*batch_size:(batch+1)*batch_size]])
        attention_mask = torch.tensor([tokens+[0]*(maxlen_sent-len(tokens)) for tokens in shuffled_attention_mask[batch*batch_size:(batch+1)*batch_size]]) 

        token_tensor = token_tensor.to('cuda')
        attention_mask = attention_mask.to('cuda')

        yield token_tensor,attention_mask


#Test dataset shared-task 2020
test_data_path = 'gdrive/My Drive/Colab Notebooks/task1_test_participant.tsv'
test_textprocess = textprocess(test_data_path)
testing_df = pd.DataFrame.from_records(test_textprocess[1:])
testing_df.columns = ['tweet_id','tweet']
print('testing_df:',len(testing_df))
#Pre-filter test dataset shared-task 2020
test_mednorm = mednorm(testing_df) 
test_mednorm_df = pd.DataFrame.from_records(test_mednorm)
test_mednorm_df.columns = ['tweet_id','tweet']
print('test_mednorm_df:',len(test_mednorm_df))
test_medword_df=testing_df[testing_df['tweet'].str.contains("drug|med|medication")]
print('test_medword_df',len(test_medword_df))
test_filter_df = pd.concat([test_mednorm_df, test_medword_df]).drop_duplicates(keep='first')
print('test_filter_df',len(test_filter_df))
test_lelf_df=pd.concat([testing_df, test_filter_df]).drop_duplicates(keep=False)
test_lelf_df.columns = ['tweet_id','tweet']
print('test_lelf_df',len(test_lelf_df))

def prob_predict(model,data):
    model.eval()
    model.cuda()
    probs = []
    batch_count = 0
    for token_tensor, attention_mask in data:
        with torch.no_grad():
            logits = model(token_tensor,token_type_ids=None,attention_mask=attention_mask)[0]
        tmp_preds = F.softmax(logits, -1)
        tmp_preds = tmp_preds.detach().cpu().numpy()
        one_prob = [each[1] for each in tmp_preds]
        probs +=  one_prob            
    print(len(probs))
    return probs

print('BERT-Large')

from transformers import BertTokenizer,BertForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
checkpoint_dir = "gdrive/My Drive/Colab Notebooks/bert-large-v4/"
probability = []
for i in range(10):
  ### load pre-train data to model_1, make sure all the files are in same folder, i.e vocab , config and bin file
  model = BertForSequenceClassification.from_pretrained(checkpoint_dir+'model '+str(i+1)+'/', cache_dir=None)
  ### define def preprocess_1 and load data to def
  encoded_tokens_Test,attention_mask_Test = preprocess(test_filter_df)
  data_test = feed_generator(encoded_tokens_Test, attention_mask_Test)
  preds_test = prob_predict(model,data_test)
  probability.append(preds_test)
  i += 1

avg_probability = np.average(np.array(probability),axis=0)
avg_predicted_label = []
for prob in avg_probability:
  if prob > 0.5:
    avg_predicted_label.append(1)
  else:
    avg_predicted_label.append(0)

print('Bio+Clinical BERT')

from transformers import AutoTokenizer,AutoModelForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
checkpoint_dir = "gdrive/My Drive/Colab Notebooks/bio-clinical-bert-v4/"
probability_1 = []
for i in range(10):
  ### load pre-train data to model_1, make sure all the files are in same folder, i.e vocab , config and bin file
  model_1 = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir+'model '+str(i+1)+'/', cache_dir=None)
  ### define def preprocess_1 and load data to def
  encoded_tokens_Test_1,attention_mask_Test_1 = preprocess(test_filter_df)
  data_test_1 = feed_generator(encoded_tokens_Test_1, attention_mask_Test_1)
  preds_test_1 = prob_predict(model_1,data_test_1)
  probability_1.append(preds_test_1)
  i += 1

avg_probability_1 = np.average(np.array(probability_1),axis=0)
avg_predicted_label_1 = []
for prob in avg_probability_1:
  if prob > 0.5:
    avg_predicted_label_1.append(1)
  else:
    avg_predicted_label_1.append(0)
"""
print('BERT-Base')

from transformers import BertTokenizer,BertForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
probability_2 = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
checkpoint_dir = "gdrive/My Drive/Colab Notebooks/bert-base-2018-v2/"
for i in range(10):
  ### load pre-train data to model_1, make sure all the files are in same folder, i.e vocab , config and bin file
  model_2 = BertForSequenceClassification.from_pretrained(checkpoint_dir+'model '+str(i+1)+'/', cache_dir=None)
  ### define def preprocess_1 and load data to def
  encoded_tokens_Test_2,attention_mask_Test_2 = preprocess(test_filter_df)
  data_test_2 = feed_generator(encoded_tokens_Test_2,attention_mask_Test_2)
  preds_test_2 = prob_predict(model_2,data_test_2)
  probability_2.append(preds_test_2)
  i += 1

avg_probability_2 = np.average(np.array(probability_2),axis=0)
avg_predicted_label_2 = []
for prob in avg_probability_2:
  if prob > 0.5:
    avg_predicted_label_2.append(1)
  else:
    avg_predicted_label_2.append(0)
"""
print('Ensemble BERT')
avg_probability_ensemble = np.average(np.array([avg_probability,avg_probability_1]),axis=0,weights=[1/2,1/2])
avg_predicted_ensemble_label = []
for prob in avg_probability_ensemble:
  if prob > 0.5:
    avg_predicted_ensemble_label.append(1)
  else:
    avg_predicted_ensemble_label.append(0)

test_filter_df['Class']=avg_predicted_ensemble_label[:len(test_filter_df)]
test_lelf_df['Class']=['0'] * len(test_lelf_df)
del test_filter_df['tweet']
del test_lelf_df['tweet']

del testing_df['tweet']
predict_test_df = pd.concat([test_filter_df, test_lelf_df])
testing_df['tweet_id']=testing_df['tweet_id'].astype(str)
predict_test_df['tweet_id'] = predict_test_df['tweet_id'].astype(str)
predict_test_df['Class'] = predict_test_df['Class'].astype(str)

testing_df.merge(predict_test_df, left_on='tweet_id', right_on='tweet_id').to_csv('gdrive/My Drive/Colab Notebooks/prediction_task1_test_v2.tsv',index=False,sep='\t', line_terminator = '\n')


     |████████████████████████████████| 3.1MB 5.4MB/s 
     |████████████████████████████████| 13.0MB 206kB/s 
     |████████████████████████████████| 6.2MB 41.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 296kB 46.8MB/s 
  Created wheel for scispacy: filename=scispacy-0.2.4-cp36-none-any.whl size=35204 sha256=aba4674c0518ad10b42a41563a3e28a1a22b26c3692e02800a35f59161290f8c
  Stored in directory: /root/.cache/pip/wheels/36/5e/7c/ba8d6b02f5cfa40735e40ccffd29852ca19ebca59393afd5f0
Successfully built scispacy
  Found existing installation: botocore 1.16.13
    Uninstalling botocore-1.16.13:
      Successfully uninstalled botocore-1.16.13
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0


     |████████████████████████████████| 500.6MB 17kB/s 
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.2.4-cp36-none-any.whl size=501343162 sha256=d64fcbcf2ea3f66c9c31fe6192a74d6268a6a2b25df30210706fa2acd88ca6d5
  Stored in directory: /root/.cache/pip/wheels/ea/ab/e5/fa667519032799529ce6a50944a82d6ae3603819cd07836aa2
Successfully built en-core-sci-lg
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 665kB 3.5MB/s 
     |████████████████████████████████| 

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/concept_aliases.json not found in cache, downloading to /tmp/tmplic46k59
Finished download, copying /tmp/tmplic46k59 to cache at /root/.scispacy/datasets/0f064d20aefab965d5772b2100f8436b3541e7d5313c76cfe5fe070902f149fe.31df9cdb04729860a81bd6c980224ed2bff582586c398d0c9b96ae4e257b9da2.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_2017_aa_cat0129.json not found in cache, downloading to /tmp/tmpu38_k661
Finished download, copying /tmp/tmpu38_k661 to cache at /root/.scispacy/datasets/13b30cd31cd37c1b52f3df6ea023061172d16e9941660e677fdbb29489af7410.4ad71d86ce780e00cab131c7e3b81acfd2f11dd80ccd61125c8bcde506f2ab8a.umls_2017_aa_cat0129.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmpani8ece1
Finished download, copying /tmp/tmpani8ece1 to cache at /root/.scispacy/datasets/21a1012c532c3a431d60895c509f5b4d45b0f

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:129: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:130: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


22080
22080
22080
22080
22080
22080
22080
22080
22080
22080
Bio+Clinical BERT



22080
22080
22080
22080
22080
22080
22080
22080
22080
22080
Ensemble BERT
